In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
modes1 = 3
width = 2
out_channels, in_channels = 2, 2

In [3]:
scale = (1 / (in_channels*out_channels))
weights1 = nn.Parameter(scale * torch.rand(in_channels, out_channels, modes1, dtype=torch.cfloat))
weights1, weights1.shape

(Parameter containing:
 tensor([[[0.0102+0.1989j, 0.0574+0.1238j, 0.0202+0.1130j],
          [0.1724+0.0178j, 0.1130+0.1229j, 0.2333+0.1410j]],
 
         [[0.1005+0.1442j, 0.2143+0.0519j, 0.1744+0.1267j],
          [0.1862+0.0849j, 0.0052+0.0722j, 0.0624+0.0029j]]],
        requires_grad=True),
 torch.Size([2, 2, 3]))

In [4]:
def get_grid(shape, device):
        batchsize, size_x = shape[0], shape[1]
        gridx = torch.tensor(np.linspace(0, 1, size_x), dtype=torch.float)
        gridx = gridx.reshape(1, size_x, 1).repeat([batchsize, 1, 1])
        return gridx.to(device)

In [5]:
w0 = nn.Conv1d(width, width, 1)
fc0 = nn.Linear(2, width)

In [6]:
x = torch.tensor([[[1], [2], [3], [4], [5], [6], [7], [8], [9] ], [[11], [12], [13], [14], [15], [16], [17], [18], [19]]])
x, x.shape

(tensor([[[ 1],
          [ 2],
          [ 3],
          [ 4],
          [ 5],
          [ 6],
          [ 7],
          [ 8],
          [ 9]],
 
         [[11],
          [12],
          [13],
          [14],
          [15],
          [16],
          [17],
          [18],
          [19]]]),
 torch.Size([2, 9, 1]))

In [7]:
grid = get_grid(x.shape, x.device)
grid, grid.shape

(tensor([[[0.0000],
          [0.1250],
          [0.2500],
          [0.3750],
          [0.5000],
          [0.6250],
          [0.7500],
          [0.8750],
          [1.0000]],
 
         [[0.0000],
          [0.1250],
          [0.2500],
          [0.3750],
          [0.5000],
          [0.6250],
          [0.7500],
          [0.8750],
          [1.0000]]]),
 torch.Size([2, 9, 1]))

In [8]:
x = torch.cat((x, grid), dim=-1)
x, x.shape

(tensor([[[ 1.0000,  0.0000],
          [ 2.0000,  0.1250],
          [ 3.0000,  0.2500],
          [ 4.0000,  0.3750],
          [ 5.0000,  0.5000],
          [ 6.0000,  0.6250],
          [ 7.0000,  0.7500],
          [ 8.0000,  0.8750],
          [ 9.0000,  1.0000]],
 
         [[11.0000,  0.0000],
          [12.0000,  0.1250],
          [13.0000,  0.2500],
          [14.0000,  0.3750],
          [15.0000,  0.5000],
          [16.0000,  0.6250],
          [17.0000,  0.7500],
          [18.0000,  0.8750],
          [19.0000,  1.0000]]]),
 torch.Size([2, 9, 2]))

In [9]:
x = fc0(x)
x, x.shape

(tensor([[[-0.2710, -0.4607],
          [-0.3003, -0.8525],
          [-0.3295, -1.2442],
          [-0.3588, -1.6359],
          [-0.3880, -2.0277],
          [-0.4173, -2.4194],
          [-0.4465, -2.8111],
          [-0.4757, -3.2029],
          [-0.5050, -3.5946]],
 
         [[ 0.0195, -4.0468],
          [-0.0097, -4.4386],
          [-0.0390, -4.8303],
          [-0.0682, -5.2220],
          [-0.0974, -5.6138],
          [-0.1267, -6.0055],
          [-0.1559, -6.3973],
          [-0.1852, -6.7890],
          [-0.2144, -7.1807]]], grad_fn=<ViewBackward0>),
 torch.Size([2, 9, 2]))

In [10]:
x = x.permute(0, 2, 1)
x, x.shape

(tensor([[[-0.2710, -0.3003, -0.3295, -0.3588, -0.3880, -0.4173, -0.4465,
           -0.4757, -0.5050],
          [-0.4607, -0.8525, -1.2442, -1.6359, -2.0277, -2.4194, -2.8111,
           -3.2029, -3.5946]],
 
         [[ 0.0195, -0.0097, -0.0390, -0.0682, -0.0974, -0.1267, -0.1559,
           -0.1852, -0.2144],
          [-4.0468, -4.4386, -4.8303, -5.2220, -5.6138, -6.0055, -6.3973,
           -6.7890, -7.1807]]], grad_fn=<PermuteBackward0>),
 torch.Size([2, 2, 9]))

In [11]:
batchsize = x.shape[0]
#Compute Fourier coeffcients up to factor of e^(- something constant)
x_ft = torch.fft.rfft(x)
x_ft, x_ft.shape

(tensor([[[ -3.4921+0.0000j,   0.1316-0.3615j,   0.1316-0.1568j,
             0.1316-0.0760j,   0.1316-0.0232j],
          [-18.2491+0.0000j,   1.7628-4.8433j,   1.7628-2.1008j,
             1.7628-1.0178j,   1.7628-0.3108j]],
 
         [[ -0.8770+0.0000j,   0.1316-0.3615j,   0.1316-0.1568j,
             0.1316-0.0760j,   0.1316-0.0232j],
          [-50.5240+0.0000j,   1.7628-4.8433j,   1.7628-2.1008j,
             1.7628-1.0178j,   1.7628-0.3108j]]], grad_fn=<FftR2CBackward0>),
 torch.Size([2, 2, 5]))

In [12]:
# Multiply relevant Fourier modes
out_ft = torch.zeros(batchsize, out_channels, x.size(-1)//2 + 1,  device=x.device, dtype=torch.cfloat)
out_ft, out_ft.shape

(tensor([[[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
 
         [[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]]]),
 torch.Size([2, 2, 5]))

In [13]:
def compl_mul1d(input, weights):
    # (batch, in_channel, x ), (in_channel, out_channel, x) -> (batch, out_channel, x)
    return torch.einsum("bix,iox->box", input, weights)

In [14]:
out_ft[:, :, :modes1] = compl_mul1d(x_ft[:, :, :modes1], weights1)
out_ft, out_ft.shape

(tensor([[[-1.8692-3.3260j,  0.6816-0.9510j,  0.5941-0.1313j,  0.0000+0.0000j,
            0.0000+0.0000j],
          [-3.9996-1.6108j,  0.4181+0.0772j,  0.1688-0.1441j,  0.0000+0.0000j,
            0.0000+0.0000j]],
 
         [[-5.0854-7.4601j,  0.6816-0.9510j,  0.5941-0.1313j,  0.0000+0.0000j,
            0.0000+0.0000j],
          [-9.5579-4.3028j,  0.4181+0.0772j,  0.1688-0.1441j,  0.0000+0.0000j,
            0.0000+0.0000j]]], grad_fn=<CopySlices>),
 torch.Size([2, 2, 5]))

In [15]:
x

tensor([[[-0.2710, -0.3003, -0.3295, -0.3588, -0.3880, -0.4173, -0.4465,
          -0.4757, -0.5050],
         [-0.4607, -0.8525, -1.2442, -1.6359, -2.0277, -2.4194, -2.8111,
          -3.2029, -3.5946]],

        [[ 0.0195, -0.0097, -0.0390, -0.0682, -0.0974, -0.1267, -0.1559,
          -0.1852, -0.2144],
         [-4.0468, -4.4386, -4.8303, -5.2220, -5.6138, -6.0055, -6.3973,
          -6.7890, -7.1807]]], grad_fn=<PermuteBackward0>)

In [16]:
#Return to physical space
x1 = torch.fft.irfft(out_ft, n=x.size(-1))
x1, x1.shape

(tensor([[[ 0.0758,  0.0958, -0.0873, -0.1917, -0.1954, -0.3024, -0.5072,
           -0.5236, -0.2333],
          [-0.3140, -0.3462, -0.4695, -0.5522, -0.5294, -0.4765, -0.4670,
           -0.4576, -0.3872]],
 
         [[-0.2816, -0.2615, -0.4447, -0.5490, -0.5527, -0.6598, -0.8645,
           -0.8809, -0.5907],
          [-0.9316, -0.9638, -1.0871, -1.1698, -1.1470, -1.0941, -1.0846,
           -1.0752, -1.0048]]], grad_fn=<FftC2RBackward0>),
 torch.Size([2, 2, 9]))

In [17]:
x2 = w0(x)
x2, x2.shape

(tensor([[[-0.3745, -0.5175, -0.6606, -0.8037, -0.9468, -1.0898, -1.2329,
           -1.3760, -1.5191],
          [-0.5439, -0.3226, -0.1014,  0.1199,  0.3412,  0.5624,  0.7837,
            1.0049,  1.2262]],
 
         [[-1.8315, -1.9745, -2.1176, -2.2607, -2.4038, -2.5468, -2.6899,
           -2.8330, -2.9760],
          [ 1.7138,  1.9351,  2.1563,  2.3776,  2.5988,  2.8201,  3.0414,
            3.2626,  3.4839]]], grad_fn=<ConvolutionBackward0>),
 torch.Size([2, 2, 9]))

In [18]:
x, x.shape

(tensor([[[-0.2710, -0.3003, -0.3295, -0.3588, -0.3880, -0.4173, -0.4465,
           -0.4757, -0.5050],
          [-0.4607, -0.8525, -1.2442, -1.6359, -2.0277, -2.4194, -2.8111,
           -3.2029, -3.5946]],
 
         [[ 0.0195, -0.0097, -0.0390, -0.0682, -0.0974, -0.1267, -0.1559,
           -0.1852, -0.2144],
          [-4.0468, -4.4386, -4.8303, -5.2220, -5.6138, -6.0055, -6.3973,
           -6.7890, -7.1807]]], grad_fn=<PermuteBackward0>),
 torch.Size([2, 2, 9]))

In [19]:
"""
    Hilbert transform convolution
"""

batchsize = x.shape[0]
N = x.size(-1)

# Compute the Fourier transform of the input
x_ft = torch.fft.fft(x, n = N)
x_ft, x_ft.shape

(tensor([[[ -3.4921+0.0000j,   0.1316-0.3615j,   0.1316-0.1568j,
             0.1316-0.0760j,   0.1316-0.0232j,   0.1316+0.0232j,
             0.1316+0.0760j,   0.1316+0.1568j,   0.1316+0.3615j],
          [-18.2491+0.0000j,   1.7628-4.8433j,   1.7628-2.1008j,
             1.7628-1.0178j,   1.7628-0.3108j,   1.7628+0.3108j,
             1.7628+1.0178j,   1.7628+2.1008j,   1.7628+4.8433j]],
 
         [[ -0.8770+0.0000j,   0.1316-0.3615j,   0.1316-0.1568j,
             0.1316-0.0760j,   0.1316-0.0232j,   0.1316+0.0232j,
             0.1316+0.0760j,   0.1316+0.1568j,   0.1316+0.3615j],
          [-50.5240+0.0000j,   1.7628-4.8433j,   1.7628-2.1008j,
             1.7628-1.0178j,   1.7628-0.3108j,   1.7628+0.3108j,
             1.7628+1.0178j,   1.7628+2.1008j,   1.7628+4.8433j]]],
        grad_fn=<FftR2CBackward0>),
 torch.Size([2, 2, 9]))

In [20]:

# Create the Hilbert transform multiplier
freqs = torch.fft.fftfreq(N).to(x.device)   # Frequency components
H = -1j * torch.sign(freqs)                 # Hilbert multiplier: -i * sign(ω)
H = H.reshape(1, 1, N)                      # Reshape for broadcasting
freqs, freqs.shape, H, H.shape

(tensor([ 0.0000,  0.1111,  0.2222,  0.3333,  0.4444, -0.4444, -0.3333, -0.2222,
         -0.1111]),
 torch.Size([9]),
 tensor([[[0.-0.j, 0.-1.j, 0.-1.j, 0.-1.j, 0.-1.j, 0.+1.j, 0.+1.j, 0.+1.j, 0.+1.j]]]),
 torch.Size([1, 1, 9]))

In [21]:
# Apply the Hilbert transform in the frequency domain
x_ht_ft = x_ft * H                           # Element-wise multiplication
x_ht_ft, x_ht_ft.shape

(tensor([[[ 0.0000+0.0000j, -0.3615-0.1316j, -0.1568-0.1316j, -0.0760-0.1316j,
           -0.0232-0.1316j, -0.0232+0.1316j, -0.0760+0.1316j, -0.1568+0.1316j,
           -0.3615+0.1316j],
          [ 0.0000+0.0000j, -4.8433-1.7628j, -2.1008-1.7628j, -1.0178-1.7628j,
           -0.3108-1.7628j, -0.3108+1.7628j, -1.0178+1.7628j, -2.1008+1.7628j,
           -4.8433+1.7628j]],
 
         [[ 0.0000+0.0000j, -0.3615-0.1316j, -0.1568-0.1316j, -0.0760-0.1316j,
           -0.0232-0.1316j, -0.0232+0.1316j, -0.0760+0.1316j, -0.1568+0.1316j,
           -0.3615+0.1316j],
          [ 0.0000+0.0000j, -4.8433-1.7628j, -2.1008-1.7628j, -1.0178-1.7628j,
           -0.3108-1.7628j, -0.3108+1.7628j, -1.0178+1.7628j, -2.1008+1.7628j,
           -4.8433+1.7628j]]], grad_fn=<MulBackward0>),
 torch.Size([2, 2, 9]))

In [22]:

# Multiply relevant frequency modes with weights
out_ft = torch.zeros(
    batchsize, out_channels, N, device=x.device, dtype=torch.cfloat
)
out_ft, out_ft.shape

(tensor([[[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
 
         [[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]]]),
 torch.Size([2, 2, 9]))

In [23]:
x_ht_ft

tensor([[[ 0.0000+0.0000j, -0.3615-0.1316j, -0.1568-0.1316j, -0.0760-0.1316j,
          -0.0232-0.1316j, -0.0232+0.1316j, -0.0760+0.1316j, -0.1568+0.1316j,
          -0.3615+0.1316j],
         [ 0.0000+0.0000j, -4.8433-1.7628j, -2.1008-1.7628j, -1.0178-1.7628j,
          -0.3108-1.7628j, -0.3108+1.7628j, -1.0178+1.7628j, -2.1008+1.7628j,
          -4.8433+1.7628j]],

        [[ 0.0000+0.0000j, -0.3615-0.1316j, -0.1568-0.1316j, -0.0760-0.1316j,
          -0.0232-0.1316j, -0.0232+0.1316j, -0.0760+0.1316j, -0.1568+0.1316j,
          -0.3615+0.1316j],
         [ 0.0000+0.0000j, -4.8433-1.7628j, -2.1008-1.7628j, -1.0178-1.7628j,
          -0.3108-1.7628j, -0.3108+1.7628j, -1.0178+1.7628j, -2.1008+1.7628j,
          -4.8433+1.7628j]]], grad_fn=<MulBackward0>)

In [24]:

out_ft[:, :, : modes1] = compl_mul1d(
    x_ht_ft[:, :, : modes1], weights1
)
out_ft

tensor([[[ 0.0000+0.0000j, -0.9510-0.6816j, -0.1313-0.5941j,  0.0000+0.0000j,
           0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
           0.0000+0.0000j],
         [ 0.0000+0.0000j,  0.0772-0.4181j, -0.1441-0.1688j,  0.0000+0.0000j,
           0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
           0.0000+0.0000j]],

        [[ 0.0000+0.0000j, -0.9510-0.6816j, -0.1313-0.5941j,  0.0000+0.0000j,
           0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
           0.0000+0.0000j],
         [ 0.0000+0.0000j,  0.0772-0.4181j, -0.1441-0.1688j,  0.0000+0.0000j,
           0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
           0.0000+0.0000j]]], grad_fn=<CopySlices>)

In [25]:

# Apply the inverse Hilbert transform multiplier
# H_inv = 1j * torch.sign(freqs)              # Inverse Hilbert multiplier: +i * sign(ω)
# H_inv = H_inv.reshape(1, 1, N)              # Reshape for broadcasting
# out_ft = out_ft * H_inv                      # Apply inverse Hilbert Transform

out_ft, out_ft.shape

(tensor([[[ 0.0000+0.0000j, -0.9510-0.6816j, -0.1313-0.5941j,  0.0000+0.0000j,
            0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
            0.0000+0.0000j],
          [ 0.0000+0.0000j,  0.0772-0.4181j, -0.1441-0.1688j,  0.0000+0.0000j,
            0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
            0.0000+0.0000j]],
 
         [[ 0.0000+0.0000j, -0.9510-0.6816j, -0.1313-0.5941j,  0.0000+0.0000j,
            0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
            0.0000+0.0000j],
          [ 0.0000+0.0000j,  0.0772-0.4181j, -0.1441-0.1688j,  0.0000+0.0000j,
            0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
            0.0000+0.0000j]]], grad_fn=<CopySlices>),
 torch.Size([2, 2, 9]))

In [26]:
        
# Return to the spatial domain
x3 = torch.fft.ifft(out_ft, n=N)       # Take the real part
x3, x3.shape

(tensor([[[-0.1203-0.1417j,  0.0302-0.1518j,  0.0925-0.0602j,  0.0685-0.0080j,
            0.0716-0.0062j,  0.1047+0.0474j,  0.0517+0.1498j, -0.1018+0.1579j,
           -0.1972+0.0128j],
          [-0.0074-0.0652j,  0.0521-0.0491j,  0.0687+0.0125j,  0.0277+0.0539j,
           -0.0165+0.0425j, -0.0242+0.0161j, -0.0203+0.0113j, -0.0356+0.0066j,
           -0.0445-0.0286j]],
 
         [[-0.1203-0.1417j,  0.0302-0.1518j,  0.0925-0.0602j,  0.0685-0.0080j,
            0.0716-0.0062j,  0.1047+0.0474j,  0.0517+0.1498j, -0.1018+0.1579j,
           -0.1972+0.0128j],
          [-0.0074-0.0652j,  0.0521-0.0491j,  0.0687+0.0125j,  0.0277+0.0539j,
           -0.0165+0.0425j, -0.0242+0.0161j, -0.0203+0.0113j, -0.0356+0.0066j,
           -0.0445-0.0286j]]], grad_fn=<FftC2CBackward0>),
 torch.Size([2, 2, 9]))

In [27]:
# x3 = hilbert(x)
# x3, x3.shape

In [33]:
from scipy.signal import hilbert, chirp
import numpy as np


In [ ]:
xn = np.random.rand(3, 4)
sin_x = np.sin(2 * np.pi * 5 * np.linspace(0, 2, 100))  # 5 Hz sine wave over 2 seconds, 100 samples
min_cos = -np.cos(2 * np.pi * 5 * np.linspace(0, 2, 100))

In [37]:
xn

array([[0.37900231, 0.05186573, 0.60913764, 0.14936949],
       [0.0279573 , 0.66431303, 0.31572433, 0.4027988 ],
       [0.40653442, 0.57942702, 0.64827321, 0.22972382]])

In [46]:
hilbert_x = hilbert(sin_x).imag

In [47]:
hilbert_x

array([-0.69235779, -0.80004521, -0.2188147 ,  0.32218905,  0.86315943,
        0.99309472,  0.81112932,  0.26021156, -0.33899771, -0.84699716,
       -0.98430948, -0.77121044, -0.22481888,  0.38172794,  0.86700978,
        0.99135349,  0.75287813,  0.20113618, -0.4088891 , -0.8771174 ,
       -0.98629525, -0.72663851, -0.16869807,  0.441493  ,  0.89319087,
        0.98516893,  0.70532176,  0.14032597, -0.46886673, -0.90465296,
       -0.97891539, -0.67998873, -0.10817521,  0.49875527,  0.91845368,
        0.97440956,  0.65691442,  0.07844625, -0.52544207, -0.9289612 ,
       -0.96641837, -0.63093986, -0.04629072,  0.55373617,  0.94075586,
        0.95951112,  0.60645382,  0.0160864 , -0.5794251 , -0.94964665,
       -0.94964665, -0.5794251 ,  0.0160864 ,  0.60645382,  0.95951112,
        0.94075586,  0.55373617, -0.04629072, -0.63093986, -0.96641837,
       -0.9289612 , -0.52544207,  0.07844625,  0.65691442,  0.97440956,
        0.91845368,  0.49875527, -0.10817521, -0.67998873, -0.97

In [49]:
min_cos

array([-1.        , -0.80527026, -0.29692038,  0.32706796,  0.82367658,
        0.99949654,  0.78605309,  0.26647381, -0.35688622, -0.84125353,
       -0.99798668, -0.76604444, -0.23575894,  0.38634513,  0.85798341,
        0.99547192,  0.74526445,  0.20480667, -0.41541501, -0.87384938,
       -0.99195481, -0.72373404, -0.17364818,  0.44406661,  0.88883545,
        0.98743889,  0.70147489,  0.14231484, -0.47227107, -0.90292654,
       -0.9819287 , -0.67850941, -0.1108382 ,  0.5       ,  0.91610846,
        0.97542979,  0.65486073,  0.07924996, -0.52722547, -0.92836793,
       -0.9679487 , -0.63055267, -0.04758192,  0.55392006,  0.93969262,
        0.95949297,  0.60560969,  0.01586596, -0.58005691, -0.95007112,
       -0.95007112, -0.58005691,  0.01586596,  0.60560969,  0.95949297,
        0.93969262,  0.55392006, -0.04758192, -0.63055267, -0.9679487 ,
       -0.92836793, -0.52722547,  0.07924996,  0.65486073,  0.97542979,
        0.91610846,  0.5       , -0.1108382 , -0.67850941, -0.98

In [50]:
-hilbert(hilbert_x).imag

array([ 0.00328432,  0.58962361,  0.95818656,  0.9417165 ,  0.57034418,
       -0.03501225, -0.61487467, -0.96712648, -0.93086354, -0.54392513,
        0.06670824,  0.63950329,  0.97509589,  0.91906998,  0.51696171,
       -0.09834036, -0.66348468, -0.98208676, -0.90634768, -0.48948105,
        0.12987677,  0.68679469,  0.98809207,  0.89270946,  0.46151084,
       -0.16128571, -0.70940985, -0.99310576, -0.87816905, -0.43307923,
        0.19253556,  0.73130739,  0.99712278,  0.86274109,  0.40421485,
       -0.22359485, -0.75246526, -1.00013909, -0.84644111, -0.37494677,
        0.2544323 ,  0.77286215,  1.00215166,  0.82928554,  0.34530446,
       -0.28501687, -0.79247752, -1.00315844, -0.81129164, -0.31531776,
        0.31531776,  0.81129164,  1.00315844,  0.79247752,  0.28501687,
       -0.34530446, -0.82928554, -1.00215166, -0.77286215, -0.2544323 ,
        0.37494677,  0.84644111,  1.00013909,  0.75246526,  0.22359485,
       -0.40421485, -0.86274109, -0.99712278, -0.73130739, -0.19

In [51]:
sin_x

array([ 0.00000000e+00,  5.92907929e-01,  9.54902241e-01,  9.45000819e-01,
        5.67059864e-01, -3.17279335e-02, -6.18158986e-01, -9.63842159e-01,
       -9.34147860e-01, -5.40640817e-01,  6.34239197e-02,  6.42787610e-01,
        9.71811568e-01,  9.22354294e-01,  5.13677392e-01, -9.50560433e-02,
       -6.66769001e-01, -9.78802446e-01, -9.09631995e-01, -4.86196736e-01,
        1.26592454e-01,  6.90079011e-01,  9.84807753e-01,  8.95993774e-01,
        4.58226522e-01, -1.58001396e-01, -7.12694171e-01, -9.89821442e-01,
       -8.81453363e-01, -4.29794912e-01,  1.89251244e-01,  7.34591709e-01,
        9.93838464e-01,  8.66025404e-01,  4.00930535e-01, -2.20310533e-01,
       -7.55749574e-01, -9.96854776e-01, -8.49725430e-01, -3.71662456e-01,
        2.51147987e-01,  7.76146464e-01,  9.98867339e-01,  8.32569855e-01,
        3.42020143e-01, -2.81732557e-01, -7.95761841e-01, -9.99874128e-01,
       -8.14575952e-01, -3.12033446e-01,  3.12033446e-01,  8.14575952e-01,
        9.99874128e-01,  